In [3]:
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import fiona
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

from datetime import datetime, timedelta
import json
import logging
import re

import bs4
import requests
import time

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

sns.set_palette(sns.diverging_palette(10, 220, sep=80, n=3, center="dark"))

sns.set_context('talk')

In [4]:
puds = gpd.read_file('../data/open_data/gis/Planned_Unit_Development_PUDs/Planned_Unit_Development_PUDs.shp')

In [5]:
census = pd.read_csv('../data/open_data/csv/Census_Block_Groups__2010.csv')

In [6]:
aff = pd.read_csv('../data/open_data/csv/Affordable_Housing.csv')

In [7]:
aff['geometry'] = aff.apply(lambda row: Point(row.X, row.Y), axis=1)

In [8]:
aff = gpd.GeoDataFrame(aff)

In [9]:
evict_csv = pd.read_csv('../data/eviction_lab/csv/block-groups.csv')

In [10]:
evict_gis = gpd.read_file('../data/eviction_lab/gis/block-groups.geojson')

In [11]:
evict_csv.GEOID = evict_csv.GEOID.astype(str)

In [12]:
evict_gis.GEOID = evict_gis.GEOID.astype(str)

In [13]:
evict0 = evict_gis[['GEOID','geometry']].merge(evict_csv, on='GEOID', how='left')

In [14]:
evict0 = evict0.loc[evict0.year == 2016]

In [15]:
evict0

,GEOID,geometry,year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
16,110010012004,"(POLYGON ((-77.07888800000001 38.946504, -77.07865 38.946617, -77.078469 38.946643, -77.07841999...",2016,12.4,"District of Columbia, District of Columbia",1405.0,0.00,188.0,30.38,1442.0,149625.0,881500.0,33.4,82.06,0.93,4.70,0.00,11.25,0.0,1.07,0.00,8.0,0.0,0.00,4.26,1,0,0
33,110010013011,"(POLYGON ((-77.042253 38.96035, -77.040616 38.959672, -77.04222900000001 38.958067, -77.042394 3...",2016,13.01.1,"District of Columbia, District of Columbia",1771.0,8.04,517.0,67.50,2126.0,129829.0,1564700.0,27.7,81.48,6.66,4.74,0.00,5.99,0.0,1.13,0.00,20.0,2.0,0.39,3.87,1,0,0
50,110010013013,"(POLYGON ((-77.061632 38.951849, -77.061639 38.949679, -77.066714 38.949683, -77.069425 38.95433...",2016,13.01.3,"District of Columbia, District of Columbia",1568.0,0.00,884.0,81.92,1701.0,103182.0,1375000.0,23.2,69.64,14.99,8.04,0.00,5.99,0.0,1.34,0.00,30.0,4.0,0.45,3.39,1,0,0
67,110010013021,"(POLYGON ((-77.056712 38.94789, -77.056456 38.94695, -77.05623 38.946297, -77.055262 38.943509, ...",2016,13.02.1,"District of Columbia, District of Columbia",2409.0,8.57,916.0,56.37,2152.0,92345.0,370800.0,32.0,60.86,7.31,10.75,0.00,17.35,0.0,2.62,1.12,58.0,5.0,0.55,6.33,1,0,0
84,110010013022,"(POLYGON ((-77.055716 38.930755, -77.05677 38.93257, -77.059243 38.936824, -77.06151699999999 38...",2016,13.02.2,"District of Columbia, District of Columbia",4549.0,0.00,2370.0,79.90,1731.0,102976.0,379500.0,27.0,76.24,8.07,10.86,0.00,3.08,0.0,1.76,0.00,45.0,11.0,0.46,1.90,1,0,0
101,110010013023,"(POLYGON ((-77.04994000000001 38.944295, -77.051303 38.941752, -77.051638 38.939726, -77.052696 ...",2016,13.02.3,"District of Columbia, District of Columbia",633.0,0.00,384.0,50.48,1768.0,118107.0,1041700.0,36.8,69.19,0.00,7.90,0.00,22.91,0.0,0.00,0.00,24.0,7.0,1.82,6.25,1,0,0
118,110010014012,"(POLYGON ((-77.07743669820709 38.9673218177274, -77.0771138374774 38.96757003193859, -77.0716409...",2016,14.01.2,"District of Columbia, District of Columbia",1160.0,0.00,41.0,6.60,2875.0,214063.0,990000.0,22.5,92.59,0.09,2.93,0.00,1.38,0.0,1.90,1.12,1.0,0.0,0.00,2.44,1,0,0
135,110010014022,"(POLYGON ((-77.070831 38.961013, -77.066238 38.961013, -77.056201 38.961013, -77.056005 38.95892...",2016,14.02.2,"District of Columbia, District of Columbia",887.0,0.00,195.0,39.96,2171.0,152778.0,839000.0,37.5,80.72,11.16,2.25,0.00,3.49,0.0,2.37,0.00,2.0,0.0,0.00,1.03,1,0,0
152,110010014023,"(POLYGON ((-77.070831 38.961013, -77.067543 38.959388, -77.066919 38.959001, -77.06603699999999 ...",2016,14.02.3,"District of Columbia, District of Columbia",1557.0,14.55,448.0,62.01,1116.0,79063.0,689900.0,29.3,48.04,15.35,16.25,1.35,13.81,0.0,2.57,2.63,30.0,8.0,1.79,6.70,1,0,0
169,110010015001,"(POLYGON ((-77.05166250431029 38.9870923275862, -77.05177 38.986949, -77.05004099999999 38.98606...",2016,15.1,"District of Columbia, District of Columbia",1654.0,0.00,28.0,5.62,NaN,210714.0,932100.0,NaN,59.92,22.97,7.19,0.00,4.29,0.0,5.62,0.00,1.0,0.0,0.00,3.57,1,0,0


In [49]:
def get_date(url, delay=.2):

    def soupify_page(url):
        headers = {"User-Agent": "my web scraping program. contact me at admin@domain.com" }
        try:
            r = requests.get(url, headers=headers, timeout=10)
            content = r.content
            soup = bs4.BeautifulSoup(content, 'html.parser')
            return soup
        except requests.exceptions.Timeout:
            pass  # handle the timeout
        except Exception as e:
            logger.critical(f'Exception making GET to {url}: {e}', exc_info=True)
            pass 

    t0 = time.time()    
    soup = soupify_page(url)
    response_delay = time.time() - t0
    request_delay = response_delay * delay
    
    while True:
        try:
            last_date = soup.find('span', {'id': re.compile(r'ExhibitDate$')}).text[-4:]
            print(f'{url[url.find("=")+1:]} {last_date}\n    Waiting {request_delay} seconds before next request...')
            time.sleep(request_delay)
            return last_date
        except (Exception):
            try:
                date = soup.find('span', {'id': 'lblHearingDate'}).text[-4:]
                print(f'{url[url.find("=")+1:]} {date}\n    Waiting {request_delay} seconds before next request...')
                time.sleep(request_delay)
                return date
            except (Exception):
                date = np.nan
                print(f'{url[url.find("=")+1:]} {date}\n    Waiting {request_delay} seconds before next request...')
                time.sleep(request_delay)
                return date 

In [54]:
puds['year'] = puds['PUD_WEB_UR'].map(lambda x: get_date(x))

80-17 1981
    Waiting 2.278643989562988 seconds before next request...
08-24C/04-25 2018
    Waiting 2.2110179901123046 seconds before next request...
01-34 2002
    Waiting 2.8983335494995117 seconds before next request...
11-25 2012
    Waiting 2.9846254348754884 seconds before next request...
12-14 2019
    Waiting 2.34266881942749 seconds before next request...
85-8 1985
    Waiting 2.886914348602295 seconds before next request...
01-36 2010
    Waiting 2.4339380264282227 seconds before next request...
08-25 2009
    Waiting 2.5856715202331544 seconds before next request...
01-12 2001
    Waiting 2.749050235748291 seconds before next request...
87-18 1988
    Waiting 2.239873123168945 seconds before next request...
86-1 1986
    Waiting 2.5696260452270505 seconds before next request...
84-13 1985
    Waiting 3.2809249877929685 seconds before next request...
83-11 1984
    Waiting 2.4824100494384767 seconds before next request...
88-34B 2017
    Waiting 3.374000358581543 seconds be

05-28L 2013
    Waiting 3.2562026023864745 seconds before next request...
05-28R 2017
    Waiting 2.5849356651306152 seconds before next request...
05-28J 2013
    Waiting 3.225241184234619 seconds before next request...
05-28F 2011
    Waiting 3.037457084655762 seconds before next request...
05-28P 2017
    Waiting 2.5201761245727536 seconds before next request...
05-28A 2008
    Waiting 4.267314147949219 seconds before next request...
05-28P 2016
    Waiting 4.310732173919678 seconds before next request...
13-09B 2017
    Waiting 4.16484375 seconds before next request...
15-05 2015
    Waiting 3.2079623222351072 seconds before next request...
15-12 2016
    Waiting 2.430047607421875 seconds before next request...
15-14 2016
    Waiting 5.650389289855957 seconds before next request...
15-01A 2016
    Waiting 3.086404895782471 seconds before next request...
15-20B 2018
    Waiting 2.4025199890136717 seconds before next request...
15-13 2019
    Waiting 2.8535997390747068 seconds before

07-26F 2015
    Waiting 2.4857205390930175 seconds before next request...
03-04 2003
    Waiting 2.627059078216553 seconds before next request...
03-24 2003
    Waiting 2.3592864990234372 seconds before next request...
04-22A 2007
    Waiting 2.5275529861450194 seconds before next request...
07-16 2008
    Waiting 2.4452482223510743 seconds before next request...
04-13C 2019
    Waiting 2.4720025062561035 seconds before next request...
06-02A 2009
    Waiting 2.326394462585449 seconds before next request...
06-13 2007
    Waiting 2.3461584091186523 seconds before next request...
06-14F 2018
    Waiting 2.3480352401733398 seconds before next request...
06-29C 2013
    Waiting 2.4366142272949216 seconds before next request...
04-19A 2007
    Waiting 2.367074489593506 seconds before next request...
06-34A 2008
    Waiting 2.5695373535156247 seconds before next request...
06-41 2007
    Waiting 2.3427623748779296 seconds before next request...
06-04G 2018
    Waiting 2.2577977180480957 sec

In [62]:
puds

,OBJECTID,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,geometry,year
0,75158,4L Associates Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=80-17,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,80-17,0,0,{95450152-37F7-41C5-B147-0C2719D9EAFA},Ward 5,"POLYGON ((-77.00471574325915 38.92462196572229, -77.00473300809519 38.92468044427013, -77.004756...",1981
1,75159,Bozutto Group,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-24C/04-25,PUD Modification July 2018,"C-2-B, R-5-B",Final,08-24C/04-25,0,0,{1FF4275D-29FF-4D38-8CFE-D7119010FAF5},Ward 5,"(POLYGON ((-76.99633034837495 38.9303615341334, -76.99688238904335 38.93036155015298, -76.998094...",2018
2,75160,National Academy of Sciences - Fifth Street,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-34,"Official Digital Zoning Map of July 1, 2010",C-3-C,Final,01-34,0,0,{F5A40354-52E1-486B-A268-201E28D938FF},Ward 2,"POLYGON ((-77.01907499933805 38.89721344569826, -77.01907474363878 38.89625726176668, -77.019730...",2002
3,75161,Baywood Hotels,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=11-25,Case # 11-25,C-3-C,Final,11-25,0,0,{0954B9AC-3079-473A-B231-AC6AC5422413},Ward 5,"POLYGON ((-76.99727251536488 38.91124895701469, -76.99664956091959 38.91146541521783, -76.996306...",2012
4,75162,3rd & M and Park Inn Associates LP,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=12-14,"Official Digital Zoning Map of July 1, 2010, Approved 3/14/14",R-5-D,Final,12-14,0,0,{D3F439EE-565C-4E75-BE94-5E922D8EC9C1},Ward 6,"(POLYGON ((-77.01538352279526 38.87774731884134, -77.01538340753703 38.87720607471513, -77.01592...",2019
5,75163,Prospect Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=85-8,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,85-8,0,0,{7281753A-8C87-427E-B290-FD5CC7299C5E},Ward 3,"POLYGON ((-77.05165473759811 38.93657780882258, -77.05164742961004 38.93655833892067, -77.051640...",1985
6,75164,Unified Communications Center,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-36,"Official Digital Zoning Map of July 1, 2010",C-2-B,Final,01-36,0,0,{CA2802A8-AE4C-4580-9C1E-05356E581877},Ward 8,"POLYGON ((-76.99367258295386 38.85433597906242, -76.99356059081177 38.85284743367416, -76.995811...",2010
7,75165,Matthews Memorial,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-25,"Official Digital Zoning Map of July 1, 2010",C-2-A,Final,08-25,0,0,{CC325C3B-83FC-4CC3-B528-C94CE258E127},Ward 8,"POLYGON ((-76.99450772873712 38.85733070854086, -76.99458188952184 38.85709882271153, -76.995684...",2009
8,75582,Nehemiah Homes at Fort Dupont,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-12,"Official Digital Zoning Map of July 1, 2010",None,Final,01-12,0,0,{A2848782-6E12-4AA2-B2B7-41E0471EDD34},Ward 7,"(POLYGON ((-76.94835414985999 38.88799472401496, -76.94808095550476 38.88772111014288, -76.94795...",2001
9,75583,Boston Properties II,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=87-18,"Official Digital Zoning Map of July 1, 2010",CR,Final,87-18,0,0,{2D661566-E6EA-498A-9DE0-26B61940093E},Ward 2,"POLYGON ((-77.05244449796034 38.90714705139236, -77.05244444792962 38.90679285039369, -77.052783...",1988


In [131]:
years = pd.DataFrame([['88-9', 0],
                              ['10-03B', 2018],
                              ['03-12X/03-13X', 2019],
                              ['94-12', 0],
                              ['06-35A', 2010],
                              ['94-16', 1994],
                              ['96-6', 1997],
                              ['92-5', 1992], 
                              ['05-280', 0],
                              ['98-18', 0],
                              ['08-08A', 2012],
                              ['09-08A', 2010],
                              ['14-04A', 2016],
                              ['05-28O', 0]],
                     columns=['PUD_CASE_N', 'year'])

In [132]:
puds0 = puds.copy()

In [133]:
years

,PUD_CASE_N,year
0,88-9,0
1,10-03B,2018
2,03-12X/03-13X,2019
3,94-12,0
4,06-35A,2010
5,94-16,1994
6,96-6,1997
7,92-5,1992
8,05-280,0
9,98-18,0


In [134]:
years0 = puds0[['PUD_CASE_N','year']].loc[~puds0['year'].isna()]

In [135]:
years_table = pd.concat([years,years0])
years_table

,PUD_CASE_N,year
0,88-9,0
1,10-03B,2018
2,03-12X/03-13X,2019
3,94-12,0
4,06-35A,2010
5,94-16,1994
6,96-6,1997
7,92-5,1992
8,05-280,0
9,98-18,0


In [136]:
years_table.drop_duplicates(subset=['PUD_CASE_N'], keep='first', inplace=True)
years_table

,PUD_CASE_N,year
0,88-9,0
1,10-03B,2018
2,03-12X/03-13X,2019
3,94-12,0
4,06-35A,2010
5,94-16,1994
6,96-6,1997
7,92-5,1992
8,05-280,0
9,98-18,0


In [137]:
puds0 = puds0.merge(years_table, on='PUD_CASE_N', how='left', suffixes=('_scraped', ''))

,OBJECTID,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,geometry,year_scraped,year


In [143]:
puds0.to_file("../data/viz/puds.shp")

In [144]:
puds0

,OBJECTID,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,geometry,year_scraped,year
0,75158,4L Associates Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=80-17,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,80-17,0,0,{95450152-37F7-41C5-B147-0C2719D9EAFA},Ward 5,"POLYGON ((-77.00471574325915 38.92462196572229, -77.00473300809519 38.92468044427013, -77.004756...",1981,1981
1,75159,Bozutto Group,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-24C/04-25,PUD Modification July 2018,"C-2-B, R-5-B",Final,08-24C/04-25,0,0,{1FF4275D-29FF-4D38-8CFE-D7119010FAF5},Ward 5,"(POLYGON ((-76.99633034837495 38.9303615341334, -76.99688238904335 38.93036155015298, -76.998094...",2018,2018
2,75160,National Academy of Sciences - Fifth Street,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-34,"Official Digital Zoning Map of July 1, 2010",C-3-C,Final,01-34,0,0,{F5A40354-52E1-486B-A268-201E28D938FF},Ward 2,"POLYGON ((-77.01907499933805 38.89721344569826, -77.01907474363878 38.89625726176668, -77.019730...",2002,2002
3,75161,Baywood Hotels,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=11-25,Case # 11-25,C-3-C,Final,11-25,0,0,{0954B9AC-3079-473A-B231-AC6AC5422413},Ward 5,"POLYGON ((-76.99727251536488 38.91124895701469, -76.99664956091959 38.91146541521783, -76.996306...",2012,2012
4,75162,3rd & M and Park Inn Associates LP,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=12-14,"Official Digital Zoning Map of July 1, 2010, Approved 3/14/14",R-5-D,Final,12-14,0,0,{D3F439EE-565C-4E75-BE94-5E922D8EC9C1},Ward 6,"(POLYGON ((-77.01538352279526 38.87774731884134, -77.01538340753703 38.87720607471513, -77.01592...",2019,2019
5,75163,Prospect Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=85-8,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,85-8,0,0,{7281753A-8C87-427E-B290-FD5CC7299C5E},Ward 3,"POLYGON ((-77.05165473759811 38.93657780882258, -77.05164742961004 38.93655833892067, -77.051640...",1985,1985
6,75164,Unified Communications Center,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-36,"Official Digital Zoning Map of July 1, 2010",C-2-B,Final,01-36,0,0,{CA2802A8-AE4C-4580-9C1E-05356E581877},Ward 8,"POLYGON ((-76.99367258295386 38.85433597906242, -76.99356059081177 38.85284743367416, -76.995811...",2010,2010
7,75165,Matthews Memorial,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-25,"Official Digital Zoning Map of July 1, 2010",C-2-A,Final,08-25,0,0,{CC325C3B-83FC-4CC3-B528-C94CE258E127},Ward 8,"POLYGON ((-76.99450772873712 38.85733070854086, -76.99458188952184 38.85709882271153, -76.995684...",2009,2009
8,75582,Nehemiah Homes at Fort Dupont,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-12,"Official Digital Zoning Map of July 1, 2010",None,Final,01-12,0,0,{A2848782-6E12-4AA2-B2B7-41E0471EDD34},Ward 7,"(POLYGON ((-76.94835414985999 38.88799472401496, -76.94808095550476 38.88772111014288, -76.94795...",2001,2001
9,75583,Boston Properties II,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=87-18,"Official Digital Zoning Map of July 1, 2010",CR,Final,87-18,0,0,{2D661566-E6EA-498A-9DE0-26B61940093E},Ward 2,"POLYGON ((-77.05244449796034 38.90714705139236, -77.05244444792962 38.90679285039369, -77.052783...",1988,1988


In [147]:
evict = pd.read_csv('../data/final_datasets0/master_puds_blocks2.csv')

In [159]:
evict

,GEOID,evict_data_year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed,geometry_Tract,ward,OBJECTID_PUD,PUD_NAME,PUD_WEB_URL,PUD_CHANGE_NARRATIVE,PUD_ZONING,PUD_STATUS,PUD_CASE_NUMBER,SHAPEAREA,SHAPELEN,GLOBALID,WARD,Name,Description,geometry_ft,Zone_Cat,CASE_ID_orig,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,TOTAL_AFFORDABLE_UNITS,MAR_WARD,Type,ANC,Total # Residential Units,% Affordable Units,Total # of 3 bedroom+ units (Fam sized units),Ownership (rental vs. condo or mix),Affordability notes (What levels of AMI% are avail),X,Y,OBJECTID_gr,LATITUDE,LONGITUDE,ADDRESS_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM,CASE_ID_update,PUD_year
0,110010001001,2016,1.1,"District of Columbia, District of Columbia",1431.0,0.00,377.0,53.29,2568.0,141792.0,1133500.0,22.1,85.67,1.47,10.76,0.00,2.10,0.0,0.00,0.00,7.0,2.0,0.53,1.86,1,0,0,"MULTIPOLYGON (((-77.068752 38.916966, -77.068145 38.918733, -77.059084 38.918313, -77.058894 38....",Ward 2,64136.0,"Cathedral Commons Partners, LLC",https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-15A,Modification 9/26/16,C-2-A,Final,08-15A,0.0,0.0,{CBE9203F-D56A-4A05-A719-DF2B1A81BFBB},Ward 3,NaN,NaN,"POLYGON ((-77.06730252242228 38.91747830399233, -77.06734037927563 38.91752768275958, -77.067352...",Commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0
1,110010001002,2016,1.2,"District of Columbia, District of Columbia",1367.0,0.00,548.0,53.75,1846.0,124808.0,796900.0,21.2,84.64,3.29,4.97,0.00,3.51,0.0,1.61,1.98,9.0,0.0,0.00,1.64,1,0,0,"MULTIPOLYGON (((-77.05126 38.910753, -77.050597 38.910004, -77.05037900000001 38.909874, -77.050...",Ward 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110010001003,2016,1.3,"District of Columbia, District of Columbia",1209.0,0.00,195.0,28.04,3018.0,237604.0,1994200.0,10.8,78.49,0.00,10.67,0.99,8.52,0.0,1.32,0.00,1.0,1.0,0.51,0.51,1,0,0,"MULTIPOLYGON (((-77.063582 38.90786, -77.064904 38.91036, -77.05714 38.910544, -77.057073 38.907...",Ward 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110010002011,2016,2.01.1,"District of Columbia, District of Columbia",3685.0,0.00,1.0,0.00,NaN,NaN,NaN,NaN,65.51,7.41,9.06,0.00,13.43,0.0,4.34,0.24,1.0,0.0,0.00,100.00,1,0,0,"MULTIPOLYGON (((-77.078008 38.905969, -77.078124 38.905983, -77.078311 38.905987, -77.0783080000...",Ward 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110010002021,2016,2.02.1,"District of Columbia, District of Columbia",1208.0,0.00,217.0,34.01,2158.0,192750.0,1330700.0,25.1,84.93,3.64,6.79,0.00,3.89,0.0,0.75,0.00,3.0,0.0,0.00,1.38,1,0,0,"MULTIPOLYGON (((-77.069382 38.917726, -77.068752 38.916966, -77.067824 38.915804, -77.065139 38....",Ward 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,110010002022,2016,2.02.2,"District of Columbia, District of Columbia",752.0,4.50,139.0,19.94,1675.0,188846.0,1163500.0,22.8,97.74,0.00,0.00,0.00,1.46,0.0,0.80,0.00,1.0,0.0,0.00,0.72,1,0,0,"MULTIPOLYGON (((-77.063582 38.90786, -77.069

In [185]:
evict.columns

Index(['GEOID', 'evict_data_year', 'name', 'parent-location', 'population',
       'poverty-rate', 'renter-occupied-households', 'pct-renter-occupied',
       'median-gross-rent', 'median-household-income', 'median-property-value',
       'rent-burden', 'pct-white', 'pct-af-am', 'pct-hispanic', 'pct-am-ind',
       'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other',
       'eviction-filings', 'evictions', 'eviction-rate',
       'eviction-filing-rate', 'low-flag', 'imputed', 'subbed',
       'geometry_Tract', 'ward', 'OBJECTID_PUD', 'PUD_NAME', 'PUD_WEB_URL',
       'PUD_CHANGE_NARRATIVE', 'PUD_ZONING', 'PUD_STATUS', 'PUD_CASE_NUMBER',
       'SHAPEAREA', 'SHAPELEN', 'GLOBALID', 'WARD', 'Name', 'Description',
       'geometry_ft', 'Zone_Cat', 'CASE_ID_orig', 'ADDRESS', 'PROJECT_NAME',
       'STATUS_PUBLIC', 'AGENCY_CALCULATED', 'AFFORDABLE_UNITS_AT_0_30_AMI',
       'AFFORDABLE_UNITS_AT_31_50_AMI', 'AFFORDABLE_UNITS_AT_51_60_AMI',
       'AFFORDABLE_UNITS_AT_61_80_AMI', 'AFFORDABLE

In [186]:
evict_mini = evict[['PUD_CASE_NUMBER', 'Zone_Cat','GEOID', 'population',
       'poverty-rate', 'renter-occupied-households', 'pct-renter-occupied',
       'median-gross-rent', 'median-household-income', 'median-property-value',
       'rent-burden', 'pct-white', 'pct-af-am', 'pct-hispanic', 'pct-am-ind',
       'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other',
       'eviction-filings', 'evictions', 'eviction-rate',
       'eviction-filing-rate', 'low-flag',
       'geometry_Tract']]

In [187]:
evict_mini = evict_mini.loc[~evict_mini.PUD_CASE_NUMBER.isna()]

In [188]:
evict_mini0 = evict_mini.copy()

In [175]:
# evict_mini0 = evict_mini0[['OBJECTID_PUD', 'Zone_Cat', 'GEOID', 'population', 'poverty-rate', 'renter-occupied-households',
#        'pct-renter-occupied', 'median-gross-rent', 'median-household-income',
#        'median-property-value', 'rent-burden', 'pct-white', 'pct-af-am',
#        'pct-hispanic', 'pct-am-ind', 'pct-asian', 'pct-nh-pi', 'pct-multiple',
#        'pct-other', 'eviction-filings', 'evictions', 'eviction-rate',
#        'eviction-filing-rate', 'low-flag', 'geometry_Tract']]

In [178]:
evict_mini0.OBJECTID_PUD = evict_mini0.OBJECTID_PUD.astype(int)

In [190]:
evict_mini0.rename(columns={"PUD_CASE_NUMBER": "PUD_CASE_N"}, inplace=True)

In [191]:
evict_mini0

,PUD_CASE_N,Zone_Cat,GEOID,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,geometry_Tract
0,08-15A,Commercial,110010001001,1431.0,0.00,377.0,53.29,2568.0,141792.0,1133500.0,22.1,85.67,1.47,10.76,0.00,2.10,0.0,0.00,0.00,7.0,2.0,0.53,1.86,1,"MULTIPOLYGON (((-77.068752 38.916966, -77.068145 38.918733, -77.059084 38.918313, -77.058894 38...."
314,08-07D,Commercial,110010001004,1202.0,0.00,260.0,34.21,3200.0,240045.0,1594700.0,21.4,77.54,0.00,16.22,0.00,6.24,0.0,0.00,0.00,2.0,1.0,0.38,0.77,1,"MULTIPOLYGON (((-77.06465794452549 38.8918438452555, -77.06729900000001 38.899211, -77.068199000..."
315,07-07C,Other/Mixed Use,110010003003,1593.0,0.00,292.0,37.98,2106.0,135547.0,916100.0,36.1,61.52,13.87,6.65,0.00,15.13,0.0,2.82,0.00,1.0,0.0,0.00,0.34,1,"MULTIPOLYGON (((-77.082035 38.919318, -77.08214700000001 38.919411, -77.082272 38.921493, -77.08..."
316,16-10,Commercial,110010003003,1593.0,0.00,292.0,37.98,2106.0,135547.0,916100.0,36.1,61.52,13.87,6.65,0.00,15.13,0.0,2.82,0.00,1.0,0.0,0.00,0.34,1,"MULTIPOLYGON (((-77.082035 38.919318, -77.08214700000001 38.919411, -77.082272 38.921493, -77.08..."
317,86-33,Other/Mixed Use,110010004001,529.0,0.00,49.0,6.60,NaN,250001.0,1362100.0,NaN,75.05,6.99,6.81,0.00,10.02,0.0,1.13,0.00,1.0,0.0,0.00,2.04,1,"MULTIPOLYGON (((-77.06639 38.931467, -77.06639699999999 38.929964, -77.066384 38.927527, -77.066..."
318,06-13,Other/Mixed Use,110010008021,1880.0,1.85,533.0,43.24,1579.0,82621.0,796300.0,29.2,58.40,14.63,10.05,0.00,11.60,0.0,5.32,0.00,14.0,4.0,0.75,2.63,1,"MULTIPOLYGON (((-77.07929739037989 38.9016231616772, -77.0822 38.901911, -77.0902 38.904211, -77..."
319,89-7,Commercial,110010008022,1273.0,0.00,119.0,18.04,3501.0,140556.0,932900.0,50.0,72.90,0.00,21.52,0.00,5.58,0.0,0.00,0.00,0.0,0.0,0.00,0.00,1,"MULTIPOLYGON (((-77.079016 38.912679, -77.07903899999999 38.911804, -77.07909100000001 38.90989,..."
320,78-30,Residential,110010009012,1688.0,0.00,168.0,25.36,1600.0,230972.0,1676900.0,30.8,82.17,4.86,7.23,0.00,4.80,0.0,0.95,0.00,1.0,0.0,0.00,0.60,1,"MULTIPOLYGON (((-77.1007 38.94891, -77.10052696966021 38.9490529381068, -77.100506 38.948781, -7..."
321,94-16,Residential,110010009013,4551.0,0.00,67.0,17.46,3501.0,241528.0,1136700.0,27.3,66.82,10.85,13.87,0.00,6.11,0.0,2.20,0.15,1.0,0.0,0.00,1.49,1,"MULTIPOLYGON (((-77.085634 38.938226, -77.085447 38.938001, -77.08558499999999 38.937662, -77.08..."
322,07-21C,Other/Mixed Use,110010010011,935.0,0.00,88.0,21.66,3501.0,136974.0,768800.0,14.0,61.71,2.35,8.66,0.00,7.49,0.0,19.79,0.00,2.0,1.0,1.14,2.27,1,"MULTIPOLYGON (((-77.09135528524489 38.9566212562027, -77.08581226828839 38.9608827084474, -77.08..."


In [193]:
puds1 = puds0.merge(evict_mini0, how='outer')

In [194]:
puds1.to_file("../data/viz/puds1.shp")

In [202]:
puds1.GEOID = puds1.GEOID.fillna(0).astype(int)

In [220]:
puds2 = puds1.copy()

In [211]:
len(str(puds1.GEOID[0]))

12

In [212]:
len(str(census.GEOID[0]))

12

In [219]:
aff

,X,Y,OBJECTID,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,LONGITUDE,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,ADDRESS_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM,geometry
0,-77.033608,38.937742,1556,Ward 4,"1410 Quincy Street Northwest, Washington, District of Columbia 20011",1408-1410 Quincy St NW,Under Construction,DHCD,1,38.937734,-77.033606,0,0,1,0,0,NaN,258989.0,397086.450000,141094.680000,1410 QUINCY STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.033608040074 38.93774225671439)
1,-77.039751,38.915825,1557,NaN,"1729 T Street Northwest, Washington, District of Columbia 20009",1729 T St NW,Under Construction,DHCD,1,NaN,NaN,0,0,1,0,0,NaN,241407.0,396552.840000,138661.890000,1729 T STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.0397506368645 38.91582498578156)
2,-77.034769,38.934611,1560,Ward 1,"1447 Oak Street Northwest, Washington, District of Columbia 20010",Rhode Island Condos,Under Construction,DHCD,2,38.934603,-77.034767,0,0,2,0,0,NaN,NaN,396985.639048,140747.098001,NaN,2019-12-30T04:00:25.000Z,POINT (-77.03476929648031 38.93461079384309)
3,-76.980438,38.869679,1561,Ward 8,"1704 R Street Southeast, Washington, District of Columbia 20020",1704 R St SE,Completed 2015 to Date,DHCD,1,38.869672,-76.980436,0,0,1,0,0,NaN,57916.0,401697.780000,133538.800000,1704 R STREET SE,2019-12-30T04:00:25.000Z,POINT (-76.98043825795355 38.86967941084557)
4,-76.974973,38.929333,1562,Ward 5,"2027 Rhode Island Avenue Northeast, Washington, District of Columbia 20018",2027 Rhode Island Ave,Under Construction,DHCD,6,38.929325,-76.974971,0,0,0,6,0,NaN,NaN,402170.219637,140160.917751,NaN,2019-12-30T04:00:25.000Z,POINT (-76.97497327753194 38.92933279501001)
5,-77.020661,38.915280,1563,Ward 6,"618 T Street Northwest, Washington, District of Columbia 20001",The Shaw,Under Construction,DHCD,5,38.915386,-77.020806,0,0,0,5,0,NaN,238431.0,398208.340000,138600.870000,618 T STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.02066131607506 38.91528023213611)
6,-77.023189,38.956558,1564,Ward 4,"721 Kennedy Street Northwest, Washington, District of Columbia 20011",WSH 7K,Under Construction,DHCD,1,38.956550,-77.023187,0,0,0,1,0,NaN,250280.0,397990.300000,143183.080000,721 KENNEDY STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.02318888466527 38.95655768321879)
7,-76.966708,38.933317,1565,Ward 5,"2911 Rhode Island Avenue Northeast, Washington, District of Columbia 20018",The Shirley,Under Construction,DHCD,13,38.933309,-76.966705,0,6,7,0,0,NaN,43463.0,402886.750000,140603.390000,2911 RHODE ISLAND AVENUE NE,2019-12-30T04:00:25.000Z,POINT (-76.96670771724307 38.93331665965651)
8,-76.987578,38.924711,1566,Ward 5,"2711 13th Street Northeast, Washington, District of Columbia 20017",Rhode Island Ave,Under Construction,DHCD,4,38.924704,-76.987576,0,0,4,0,0,NaN,305797.0,401077.320000,139647.670000,2711 13TH STREET NE,2019-12-30T04:00:25.000Z,POINT (-76.98757842598094 38.92471131644334)
9,-77.032483,38.918996,1567,Ward 1,"1400 W Street Northwest, Washington, District of Columbia 20009",1400 W ST NW,Under Construction,DHCD,20,38.918988,-77.032481,0,0,20,0,0,NaN,NaN,397183.222566,139013.626426,NaN,2019-12-30T04:00:25.000Z,POINT (-77.03248329481296 38.91899579078174)


In [266]:
!pip install rtree

  Using cached https://files.pythonhosted.org/packages/11/1d/42d6904a436076df813d1df632575529991005b33aa82f169f01750e39e4/Rtree-0.9.3.tar.gz
    ERROR: Command errored out with exit status 1:
     command: /Users/flatironschool/anaconda3/envs/learn-env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/jb/dxj2187n3zl1jmqn02b6ccqw0000gn/T/pip-install-q6k3wo3o/rtree/setup.py'"'"'; __file__='"'"'/private/var/folders/jb/dxj2187n3zl1jmqn02b6ccqw0000gn/T/pip-install-q6k3wo3o/rtree/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/jb/dxj2187n3zl1jmqn02b6ccqw0000gn/T/pip-install-q6k3wo3o/rtree/pip-egg-info
         cwd: /private/var/folders/jb/dxj2187n3zl1jmqn02b6ccqw0000gn/T/pip-install-q6k3wo3o/rtree/
    Complete output (15 lines):
    Traceback (most recent call last):
      File "<string>

In [235]:
gpd.sjoin(puds2, aff, op='intersects', how='left')

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


AttributeError: 'NoneType' object has no attribute 'intersection'

In [ ]:
gpd.sjoin(puds0)